<a href="https://colab.research.google.com/github/isammartins/separador-tabelas/blob/main/teste_separando_tabelas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Separação e Visualização de Tabelas Estruturadas
- Este projeto tem como objetivo **extrair e organizar múltiplas tabelas embutidas em um único bloco de texto**, onde os dados estão separados por `|` (pipe), mas não seguem um formato padrão de arquivos CSV ou Excel.
- Essa abordagem é útil quando recebemos **dados colados em um único campo** (como relatórios de sistemas antigos, PDFs convertidos, ou respostas de APIs mal formatadas), e precisamos transformá-los em **estruturas tabulares organizadas** para análise posterior.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import pandas as pd
with open("/content/drive/MyDrive/ESTRUTURA.CSV", encoding="utf-8") as f:
    linhas = [linha.strip() for linha in f.readlines()]

## Abrir o arquivo
- `open("arquivo.csv", ...)` → abre o arquivo pra leitura.
- `encoding='utf-8'` → diz ao Python qual "alfabeto digital" usar pra interpretar os caracteres.
- `as f` → dá o nome f ao arquivo aberto, como um apelido temporário.
- `.readlines()` lê todas as linhas como uma lista de strings.
- `.strip()` remove quebras de linha (\n) e espaços extras.
- linhas vira uma lista com todas as linhas já limpinhas.

In [6]:
tabelas = {}
nome_tabela = None
dados = []

- tabelas: dicionário onde vamos guardar cada mini-tabela separada.
- nome_tabela: salva o nome atual da tabela (tipo "VENDAS"). Porque antes de começar a ler qualquer tabela, você ainda não sabe qual é o nome da primeira. Então None é só um valor neutro, uma "posição de espera". É tipo: "Por enquanto não estou dentro de nenhuma tabela, mas assim que eu ver um nome como VENDAS, eu guardo ali."
- dados: armazena temporariamente o cabeçalho + as linhas de dados da tabela atual.


In [7]:
for linha in linhas:
    linha = linha.strip()

    if "|" not in linha:
        if nome_tabela and dados:
            tabelas[nome_tabela] = dados

        nome_tabela = linha
        dados = []
    else:
        dados.append(linha.split("|"))

## Explicando o código
`for linha in linhas:` ➡️ Loop que percorre cada linha do arquivo lido.

    `linha = linha.strip()` ➡️ Remove espaços em branco no início e no fim da linha, inclusive o \n (quebra de linha).

    `if "|" not in linha:`➡️ Aqui tu faz uma checagem:
Se não tem o caractere |, é porque essa linha não é um dado, mas sim um título de tabela (tipo: CLIENTES, PEDIDOS, etc).
        
        `if nome_tabela and dados:` ➡️ Verifica se:
- Já tem um nome de tabela anterior (nome_tabela)
- Já tem dados acumulados (dados)
Se sim, é hora de salvar os dados que estavam sendo coletados até agora.

            `tabelas[nome_tabela] = dados` ➡️ Guarda os dados no dicionário tabelas, usando o nome da tabela como chave

        `nome_tabela = linha` ➡️ Atualiza o nome da tabela atual.
        `dados = []` ➡️ Reinicia a lista de dados pra nova tabela começar do zero.
    `else:
        dados.append(linha.split("|"))` ➡️ Divide a linha nas colunas (pelo separador |) e adiciona aos dados.

In [8]:
if nome_tabela and dados:
    tabelas[nome_tabela] = dados

Se nome_tabela não for vazio nem None nem False (ou seja, existir) e dados também existir (não estiver vazio), aí sim adiciona no dicionário tabelas a entrada com chave nome_tabela e valor dados. Isso serve pra garantir que só salva entradas válidas no dicionário, evitando lixo.

In [9]:
for nome, conteudo in tabelas.items():
    df= pd.DataFrame(conteudo[1:], columns=conteudo[0])
    df.to_csv(f"{nome}.csv", sep="|", index= False)
    print(f"tabela '{nome}' salva como {nome}.csv")

tabela 'VENDAS' salva como VENDAS.csv
tabela 'DETALHE_VENDA' salva como DETALHE_VENDA.csv
tabela 'PRODUTOS' salva como PRODUTOS.csv


Esse for passa por cada item do dicionário tabelas, onde:

- nome é a chave (tipo "VENDAS")

- conteudo é o valor (uma lista de listas, sendo a primeira sublista os nomes das colunas)

Depois, cria um DataFrame do Pandas com:

- conteudo[0] → os nomes das colunas (header)

- conteudo[1:] → os dados em si (linhas seguintes)

- Salva esse DataFrame como um arquivo CSV chamado nome.csv, usando separador pipe | e sem índice.

Imprime uma mensagem confirmando que salvou.

In [10]:
for nome, conteudo in tabelas.items():
    print(f"\n--- Tabela: {nome} ---")
    df = pd.DataFrame(conteudo[1:], columns=conteudo[0])


--- Tabela: VENDAS ---

--- Tabela: DETALHE_VENDA ---

--- Tabela: PRODUTOS ---


- `for nome, conteudo in tabelas.items():`

Percorre todas as entradas do dicionário tabelas.

Nome é o nome da tabela (ex: "VENDAS")

conteudo é a lista de listas com os dados (onde a primeira lista são os cabeçalhos)

- `print(f"\n--- Tabela: {nome} ---")`

Imprime no terminal um título bonito.

O \n antes é pra pular uma linha e deixar visual mais clean.

- `df = pd.DataFrame(conteudo[1:], columns=conteudo[0])`
Cria um DataFrame do Pandas, onde:

 conteudo[0] são os nomes das colunas

 conteudo[1:] são as linhas de dados




In [11]:
from IPython.display import display, Markdown

for nome, conteudo in tabelas.items():
    df = pd.DataFrame(conteudo[1:], columns=conteudo[0])
    display(Markdown(f"### Tabela: {nome}"))
    display(df)

### Tabela: VENDAS

,ID_VENDA,DATA_VENDA,VALOR_VENDA
0,1,01/11/2019,1000
1,2,01/11/2019,2000
2,3,01/11/2019,3000


### Tabela: DETALHE_VENDA

,ID_VENDA,COD_PRODUTO,VALOR_UNIT,QTD,VAL_DESC
0,1,1,100,5,0
1,1,2,50,5,0
2,1,3,10,25,0
3,2,1,100,10,0
4,2,2,50,10,0
5,2,3,10,50,0
6,3,1,100,20,0
7,3,2,50,10,0
8,3,3,10,50,0


### Tabela: PRODUTOS

,COD_PRODUTO,NOME_PRODUTO
0,1,TECLADO SEM FIO
1,2,MOUSE
2,3,MOUSE PAD


`from IPython.display import display, Markdown`

Importa ferramentas do Jupyter pra:

- display() → mostrar conteúdo formatado (como tabelas e markdown)

- Markdown() → formatar texto como título bonitinho no notebook

`for nome, conteudo in tabelas.items():`

Pega cada tabela que separada antes:

- nome é o nome da tabela (ex: "VENDAS")

- conteudo é uma lista de listas: a primeira linha são os nomes das colunas, e o resto são os dados

`df = pd.DataFrame(conteudo[1:], columns=conteudo[0])`

Cria um DataFrame (tabela do Pandas):

- conteudo[0] → cabeçalhos (nomes das colunas)

- conteudo[1:] → os dados (linhas da tabela)

`display(Markdown(f"### Tabela: {nome}"))`

Mostra um título em negrito e bonito no Jupyter, o ### é título nível 3 do Markdown.

### OBS: diferença entre display e print

print() → básico, serve pra texto. Usa quando quiser ver rapidamente algo simples.

display() → Mostra tudo mais bonito, tabelado, renderizado e com múltiplos outputs numa célula só.